In [2]:
from susy_qm import ansatze
from susy_qm.ansatze import gate_list_from_ansatz
import pennylane as qml
from pennylane import numpy as np

import git
repo_path = git.Repo('.', search_parent_directories=True).working_tree_dir

import pandas as pd
import json
import os

In [14]:
def label_to_gate_count(label):
    parts = label.split("_")
    #total = 0
    #for p in parts:
    #    num = ''.join([c for c in p if c.isdigit()])
    #    total += int(num) if num else 1
    total = parts[-1]
    return int(total)

In [16]:
potentials = ['AHO']
cutoffs = [8,16]

for potential in potentials:
    for cutoff in cutoffs:

        print(f"Running for {potential} potential and {str(cutoff)} cutoff")

        folder_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\VQE\AnsatzSteps\Grouped\10000\{}\{}".format(potential,cutoff))
        folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

        num_qubits = int(np.log2(cutoff)+1) 
        ansatz_name = f"CQAVQE_{potential}{cutoff}_exact"
        ansatz = ansatze.get(ansatz_name)
        num_params = ansatz.n_params

        dev = qml.device("default.qubit", wires=num_qubits)
        @qml.qnode(dev)
        def circuit(params):
            ansatz(params, num_qubits)
            return qml.state()

        params = np.array([0]*num_params)
        gates = gate_list_from_ansatz(ansatz, params, num_qubits)

        gate_list = {}
        counter = 1 
        all_labels = ""
        for item in gates:
            if item["gate"] == "BasisState":
                continue

            gate = item["gate"]
            wires = item["wires"]

            label = f"{gate}{wires}"
            if counter == 1:
                all_labels = label
            else:
                all_labels = all_labels + ", " + label

            gate_list[str(counter)] = all_labels
            counter+=1

        avqe_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\qml.expval\Adaptive-VQE\StatevectorFiles\{}\data_{}.txt".format(potential,cutoff))

        with open(avqe_path, "r", encoding="utf-8") as f:
            text = f.read()

        json_part = text[: text.rfind("}") + 1]
        data = json.loads(json_part)

        energy_diffs = data["energy_diffs"]
        energy_changes = data["energy_change"]
        step_energies = data["best_energy_list_reduced"]

        all_data = []

        for f in folders:

            gate_count = label_to_gate_count(f)

            data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
            
            with open(data_path, 'r') as file:
                data = json.load(file)

            success = sum(data['success'])
            exact_e = np.min(data['exact_eigenvalues'])
            converged_runs = sum(data['success'])
            mean_evals = int(np.round(np.mean(data['num_evaluations'])))

            min_e = np.min(data['results'])
            delta_min_e = abs(exact_e - np.min(data['results']))
            median_e = np.median(data['results'])
            delta_median_e = abs(exact_e - np.median(data['results']))
            
            time_taken = data['total_VQE_time']

            ansatz_step = gate_list[str(gate_count)]

            data_dict = {"gate list": ansatz_step, 
                        "avqe step": gate_count,
                        'Converged Runs': f"{converged_runs}/{len(data['results'])}", 
                        "step_energies": step_energies[gate_count-1],
                        "CQAVQE Energy Diff": energy_diffs[gate_count-1],
                        "CQAVQE Energy change": energy_changes[gate_count-1],
                        "Mean Evals": mean_evals,
                        "Exact E": exact_e,
                        "Min E": min_e,
                        "Delta min E": delta_min_e,
                        "Median E": median_e,
                        "Delta median E": delta_median_e}
            
            all_data.append(data_dict)

        df = pd.DataFrame(all_data).sort_values("Delta median E", ascending=False)
        df.to_excel(os.path.join(folder_path, "ansatz_steps.xlsx"), index=False)

Running for AHO potential and 8 cutoff
Running for AHO potential and 16 cutoff


In [72]:
df = df.sort_values("avqe step", ascending=True)
df

,gate list,avqe step,Converged Runs,step_energies,CQAVQE Energy Diff,CQAVQE Energy change,Mean Evals,Exact E,Min E,Delta min E,Median E,Delta median E
6,RY[3],1,100/100,0.709178,7.103452e-01,0.000000,531,-0.001167,-0.023966,0.022799,0.893901,0.895068
0,"RY[3], RY[2]",2,100/100,0.684881,6.860479e-01,0.024297,1006,-0.001167,-0.024479,0.023312,1.400638,1.401805
1,"RY[3], RY[2], RY[1]",3,100/100,0.038341,3.950764e-02,0.646540,1402,-0.001167,-0.022854,0.021688,2.129380,2.130547
5,"RY[3], RY[2], RY[1], CRY[3, 2]",4,100/100,0.018906,2.007275e-02,0.019435,1753,-0.001167,-0.024441,0.023274,0.914789,0.915956
2,"RY[3], RY[2], RY[1], CRY[3, 2], CRY[3, 1]",5,100/100,0.010364,1.153112e-02,0.008542,2086,-0.001167,-0.023450,0.022283,7.848361,7.849528
3,"RY[3], RY[2], RY[1], CRY[3, 2], CRY[3, 1], CRY...",6,100/100,0.009265,1.043167e-02,0.001099,2450,-0.001167,-0.015646,0.014479,8.863231,8.864398
4,"RY[3], RY[2], RY[1], CRY[3, 2], CRY[3, 1], CRY...",7,100/100,-0.001167,6.955844e-13,0.010432,2738,-0.001167,-0.024429,0.023262,8.344045,8.345212


In [4]:
potentials = ['DW','AHO']
cutoffs = [8,16]

for potential in potentials:
    for cutoff in cutoffs:

        print(f"Running for {potential} potential and {str(cutoff)} cutoff")

        folder_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\VQE\AnsatzSteps\{}\{}{}".format(potential,potential,cutoff))
        folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

        num_qubits = int(np.log2(cutoff)+1) 
        ansatz_name = f"CQAVQE_{potential}{cutoff}_exact"
        ansatz = ansatze.get(ansatz_name)
        num_params = ansatz.n_params

        params = np.array([0]*num_params)
        gates = gate_list_from_ansatz(ansatz, params, num_qubits)

        gate_list = {}
        counter = 1 
        all_labels = ""
        for item in gates:
            if item["gate"] == "BasisState":
                continue

            gate = item["gate"]
            wires = item["wires"]

            label = f"{gate}{wires}"
            if counter == 1:
                all_labels = label
            else:
                all_labels = all_labels + ", " + label

            gate_list[str(counter)] = all_labels
            counter+=1

        avqe_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\qml.expval\Adaptive-VQE\StatevectorFiles\{}\data_{}.txt".format(potential,cutoff))

        with open(avqe_path, "r", encoding="utf-8") as f:
            text = f.read()

        json_part = text[: text.rfind("}") + 1]
        data = json.loads(json_part)

        energy_diffs = data["energy_diffs"]
        energy_changes = data["energy_change"]
        step_energies = data["best_energy_list_reduced"]

        all_data = []

        for f in folders:

            gate_count = label_to_gate_count(f)

            data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
            
            with open(data_path, 'r') as file:
                data = json.load(file)

            success = sum(data['success'])
            exact_e = np.min(data['exact_eigenvalues'])
            converged_runs = sum(data['success'])
            mean_evals = int(np.round(np.mean(data['num_evaluations'])))

            min_e = np.min(data['results'])
            delta_min_e = abs(exact_e - np.min(data['results']))
            median_e = np.median(data['results'])
            delta_median_e = abs(exact_e - np.median(data['results']))
            
            time_taken = data['total_VQE_time']

            ansatz_step = gate_list[str(gate_count)]

            data_dict = {"Potential": potential,
                        r"$\Lambda$": cutoff,
                        "avqe step": gate_count,
                        "gate list": ansatz_step, 
                        #'Converged Runs': f"{converged_runs}/{len(data['results'])}", 
                        "VQE-None": step_energies[gate_count-1],
                        "VQE-None-Diff": energy_diffs[gate_count-1],
                        #"CQAVQE Energy change": energy_changes[gate_count-1],
                        #"Mean Evals": mean_evals,
                        #"Min E": min_e,
                        #"Delta min E": delta_min_e,
                        "VQE-10k": median_e,
                        "VQE-10K-Diff": delta_median_e,
                        #"Exact E": exact_e,
                        }
            
            all_data.append(data_dict)

        df = pd.DataFrame(all_data).sort_values("avqe step", ascending=True)
        df.to_latex(os.path.join(folder_path, f"{potential}{cutoff}.tex"), index=False, float_format="%.6f")

Running for DW potential and 8 cutoff
Running for DW potential and 16 cutoff
Running for AHO potential and 8 cutoff
Running for AHO potential and 16 cutoff
